In [1]:
from a_protocol_0.utils.decorators import log

class Test():
    
    @log
    def test_log(self, a, b, c=2, d=None):
        print("exec test_log")
        
t = Test()

t.test_log("toto", "titi", c="42")

AttributeError: 'NoneType' object has no attribute 'log_debug'

In [ ]:
import time

start_at = time.time()
time.sleep(1.2323)
end_at = time.time()

print("%.3f" % (start_at - end_at))

In [ ]:
import traceback

try:
    raise RuntimeError("hello")
except (Exception, RuntimeError):
    print(traceback.format_exc().splitlines())


In [ ]:
"toto".splitlines()

In [ ]:
"%s terminated in %.3fs" % ("toto", 2.3232)

In [ ]:
from functools import partial
from collections import deque, Iterable


class Toto():
    pass

t = Toto()
t.titi = 2

setattr(t, "titi", 3)

f = partial(setattr, t, "titi", 4)
f()

print(t.titi)



isinstance(lambda: True, Iterable)

In [ ]:
from collections import deque, Iterable
steps = deque([1, 2, 3])
steps[-1]

print(0 or 0)

In [6]:
from functools import partial
from a_protocol_0.sequence.Sequence import Sequence
from __future__ import print_function

from a_protocol_0 import Protocol0, EmptyModule
p0 = Protocol0(EmptyModule(is_false=False), init_song=False)

def seq_creator(i):
    if i == 0:
        return None
    return Sequence().add(partial(seq_creator, i - 1), name="seq %s" % i).done()

def check_function(res):
    print("check and return %s" % res)
    return res

def check_seq(res):
    seq = Sequence(name="check seq")
    seq.add(partial(check_function, res))
    return seq.done()

def inner_print_seq(message):
    seq = Sequence(name="inner print seq")
    seq.add(lambda: print(message))
    return seq.done()

def print_seq(message):
    seq = Sequence(name="print seq")
    seq.add(partial(inner_print_seq, message))
    return seq.done()

with p0.component_guard():
#     seq = Sequence(name="test seq")
#     seq.add(partial(seq_creator, 2))
#     seq.done()()
#     check_seq(True)()
#     print("------")
    
#     seq = Sequence(name="main").add(partial(print_seq, "titi messaged"), do_if=partial(check_seq, True))
#     seq.done()()

    seq = p0.protocol0_song.select_track(EmptyModule(), sync=True)
    print(seq._auto_start)
    
# p0.actionManager.action_scroll_tracks(go_next=True)



P0 - info: Protocol0 script loaded
P0 - info: [seq SongActionMixin.select_track] : exec [step partial.setattr (and wait for listener call : function.callback_caller) (has_if)]
P0 - debug: [step partial.setattr (and wait for listener call : function.callback_caller) (has_if)] condition returned False (sequence\SequenceStep.py:100 in _terminate_if_condition)
P0 - info: [seq SongActionMixin.select_track] : exec [step wait 1 (and wait 1)]
P0 - info: [seq SongActionMixin.select_track] terminated successfully
True


In [8]:
set([1, 1, 2])

{1, 2}